## Project Brief: Job posting ingestion pipeline


### CoNVO

**Context:** Bloc is a career services management platform that builds smart career and data management tools for job-seekers and the organizations serving them. In particular, Bloc seeks to provide and facilitate access to tools for effectively presenting job-seekers' credentials and matching employers' job postings, and thereby improve outcomes.

**Need:** Job-seekers need help finding job postings that are aligned with their skills, goals, salary, and location (among other constraints), since that helps to reduce time spent filtering out bad matches and improve the likelihood of a successful outcome. Typical "fuzzy search" by job title, maybe with structured filters layered on top, performs decently well, but does take time. Unfortunately, an outside consumer of job postings like Bloc generally has more restricted options in terms of a jobs search.

**Vision:** Automated extraction of key information from job postings, and perhaps classification of postings into relevant categories, pulled from external data sources, in order to better match job postings with job-seekers.

**Outcome:** A standalone, proof-of-concept process for extracting key job posting information (and categorizing it into relevant classes), and returning it as structured data, complete with unit tests and documentation on expected usage, limitations, and potential improvements.


### Data Summary

A collection of ~575 job postings for tech+data roles in the NYC area have been fetched from the following external APIs:

- [GitHub Jobs API](https://jobs.github.com/api)
- [Indeed.com API](https://opensource.indeedeng.io/api-documentation/)
- [TheMuse.com API](https://www.themuse.com/developers/api/v2)

The APIs all return JSON-formatted data, but with a variety of fields and level of detail. All jobs include structured data for the job title, location, company, and URL, as well as a field containing either all or a short snippet of the job description as text data. These descriptions generally contain information on required / desired applicant qualifications, expected responsibilities and tasks, company missions and benefits, and more -- but snippets don't consistently contain the most important details.


### Proposed Methodology

The first and simplest task is to transform the structured data from each of the data sources into a consistent representation; in practice, this will entail renaming fields and parsing values (e.g. dates) into a standardized form. The second task is to extract as much relevant information from the free text job descriptions as possible -- key skills, technological expertise, education requirements, etc. -- in a form that allows for matching against job-seekers' skills, expertise, education. A combination of regular expressions, fuzzy string matching, and dictionary/gazetteer lookups, and other basic text processing will hopefully be enough to reliably extract info. The final, possibly optional task is to train text categorizers to predict classes of interest based on the descriptions' text: company culture? flexibility of work-life balance? Given the small size of the dataset, a simple bag-of-words-based approach may be more likely to succeed than, say, a deep learning model.


### Definitions of Success

- **Baseline:** A function that accepts a job posting from any of the three listed data sources and returns standardized, structured data for key job metadata (title, location, company, and posting URL), plus any available information on skills/expertise/education required or desired for the role. The quality of the extracted values may be messy or not fully structured, but shouldn't contain values from other fields. This function should have basic unit tests and documentation.
- **Target:** A function that accepts a job posting from any of the three listed data sources and returns standardized, structured data for key job metadata and available information on required/desired job qualifications (see Baseline), plus any available information on expected responsibilities and company mission/benefits. The quality of the extracted values should be mostly structured -- not large blobs of text, just the most informative bits -- and shouldn't contain values from other fields. This function should have unit tests covering a variety of expected scenarios and good documentation.
- **Stretch:** A function as described in Target, plus trained text categorization models for predicting one or more classes of interest. Text preprocessing and model structure must be documented, as well as validation results for model performance.


### Risks

It's possible that the information included in / extracted from job postings is too complex or varied for sufficiently accurate rules-based parsing, in which case a more sophisticated (ML- or DL-based) approach would be necessary, albeit impractical owing to time and data constraints. It's also possible that a rules-based approach is feasible, but too difficult / large a task for a single day's work.

Similarly, job categories that Bloc would find most useful may not be feasible given the size and limitations of the available training dataset. The challenge here is finding categorizations that are still useful but not too abstract/complex for a machine to learn with sufficient accuracy in a single day's work.

## Source Code

In [1]:
%load_ext watermark

### Dataset Generation

In [2]:
import io
import json
import os
import random

import requests

In [3]:
%watermark -v -iv

requests 2.22.0
json     2.0.9
CPython 3.7.4
IPython 7.8.0


In [4]:
data_dir = "/Users/burtondewilde/Desktop/datakind/bloc/msvdd_Bloc/data/postings"
role_queries = [
    "web developer",
    "front-end developer",
    "back-end developer",
    "full-stack developer",
    "python",
    "data scientist",
    "data engineer",
]
category_queries = ["Engineering", "Data Science"]

In [5]:
def dedupe_results(results, dupe_key):
    """
    Args:
        results ((List[dict]))
        dupe_key (str)
    
    Returns:
        List[dict]
    """
    unique_results = []
    seen_keys = set()
    for result in results:
        if result[dupe_key] in seen_keys:
            continue
        else:
            seen_keys.add(result[dupe_key])
            unique_results.append(result)
    return unique_results


def chunk_items(items, chunk_size):
    """
    Yield successive chunks of items.
    
    Args:
        items (list or tuple)
        chunk_size (int)
    
    Yields:
        list or tuple
    """
    for i in range(0, len(items), chunk_size):
        yield items[i : min(i + chunk_size, len(items))]
        
    
def save_jobs_to_disk(data, filepath):
    """
    Args:
        data (List[dict])
        filepath (str)
    """
    with io.open(filepath, mode="wt", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

#### GitHub Jobs API

In [6]:
def search_github_jobs(*, query, location=None, limit=None):
    """
    Args:
        query (str): A search term, such as "ruby" or "java".
        location (str): Like "New York, NY".
        limit (int)
        
    Returns:
        List[dict]
    """
    base_url = "https://jobs.github.com/positions.json"
    params = {"description": query}
    if location:
        params["location"] = location
    results = []
    i = 0
    while True:
        page_params = params.copy()
        page_params["page"] = i
        try:
            response = requests.get(base_url, params=page_params)
            response.raise_for_status()
        except requests.RequestException as e:
            print(e)
            break
        page_results = response.json()
        results.extend(page_results)
        i += 1
        if len(page_results) < 50:
            break
        elif limit is not None and len(results) > limit:
            break
    if limit is not None:
        return results[:limit]
    else:
        return results

In [7]:
all_search_results = []
for query in role_queries:
    print("query:", query)
    try:
        results = search_github_jobs(query=query, location="New York, NY", limit=100)
        all_search_results.extend(results)
    except Exception:
        break

# deduplicate the list of search results
github_jobs = dedupe_results(all_search_results, "id")
print("# jobs fetched =", len(github_jobs))
github_jobs[:1]

query: web developer
query: front-end developer
query: back-end developer
query: full-stack developer
query: python
query: data scientist
query: data engineer
# jobs fetched = 20


[{'id': '3e378b91-7cc0-486e-b18d-518ab29569e3',
  'type': 'Full Time',
  'url': 'https://jobs.github.com/positions/3e378b91-7cc0-486e-b18d-518ab29569e3',
  'created_at': 'Wed Sep 11 18:59:59 UTC 2019',
  'company': 'Simon & Schuster ',
  'company_url': 'https://www.simonandschuster.com/',
  'location': 'New York, NY',
  'title': 'Lead Software Engineer',
  'description': '<p>Simon &amp; Schuster is seeking a Lead Software Engineer to join a rapidly growing team focused on impacting the world of publishing through research and innovation. Working with a team of data scientists, data engineers, designers, and domain experts, you will be involved in rapidly prototyping, developing, and deploying the platforms that put insights and information into the hands of decision-makers.</p>\n<p>From databases to serverless applications, you will be designing, deploying , and managing the systems that connect our real-world data to the books and authors that inform and entertain our world. As a Full

In [8]:
save_jobs_to_disk(github_jobs, os.path.join(data_dir, "github_jobs.json"))

#### Indeed.com API

In [9]:
def search_indeed_jobs(publisher_id, user_ip, *, query, location=None, limit=None):
    """
    Args:
        publisher_id (str)
        user_ip (str)
        query (str): A search term, such as "ruby" or "java".
        location (str): Like "New York, NY".
        limit (int)
    
    Returns:
        List[dict]
    """
    base_url = "http://api.indeed.com/ads/apisearch"
    results_per_page = 25
    params = {
        "publisher": publisher_id,
        "userip": user_ip,
        "useragent": "blocbot python/v1",
        "v": 2,
        "format": "json",
        "filter": 1,
        "limit": results_per_page,  # not the same as `limit` kwarg
    }
    params["q"] = query
    if location:
        params["l"] = location
    start = 0
    results = []
    while True:
        page_params = params.copy()
        page_params["start"] = start
        try:
            response = requests.get(base_url, params=page_params)
            response.raise_for_status()
        except requests.RequestException as e:
            print(e)
            break
        page_results = response.json().get("results", [])
        results.extend(page_results)
        start += results_per_page
        if len(page_results) < results_per_page:
            break
        elif limit is not None and len(results) > limit:
            break
    if limit is not None:
        results = results[:limit]
    
    detailed_results = []
    job_keys = [result["jobkey"] for result in results]
    base_url = "http://api.indeed.com/ads/apigetjobs"
    params = {
        "publisher": publisher_id,
        "userip": user_ip,
        "v": 2,
        "format": "json",
    }
    for job_keys_chunk in chunk_items(job_keys, 10):
        chunk_params = params.copy()
        chunk_params["jobkeys"] = ",".join(job_key for job_key in job_keys_chunk)
        try:
            response = requests.get(base_url, params=chunk_params)
            response.raise_for_status()
        except requests.RequestException as e:
            print(e)
            break
        chunk_results = response.json().get("results", [])
        detailed_results.extend(chunk_results)
    return detailed_results

In [10]:
with io.open("/Users/burtondewilde/.bloc/indeed", mode="rt") as f:
    publisher_id = f.read().strip()

In [11]:
user_ip = requests.get("https://api.ipify.org").text

In [12]:
all_search_results = []
for query in role_queries:
    print("query:", query)
    try:
        results = search_indeed_jobs(publisher_id, user_ip, query=query, location="New York, NY", limit=100)
        all_search_results.extend(results)
    except Exception:
        break

# deduplicate the list of search results
indeed_jobs = dedupe_results(all_search_results, "jobkey")
print("# jobs fetched =", len(indeed_jobs))
indeed_jobs[:1]

query: web developer
query: front-end developer
query: back-end developer
query: full-stack developer
query: python
query: data scientist
query: data engineer
# jobs fetched = 443


[{'jobtitle': 'Front-End Developer',
  'company': 'The New Republic',
  'city': 'New York',
  'state': 'NY',
  'country': 'US',
  'language': 'en',
  'formattedLocation': 'New York, NY',
  'source': 'The New Republic',
  'date': 'Sat, 14 Sep 2019 15:30:05 GMT',
  'snippet': 'The front-end developer will identify, design, and build editorial initiatives—from news projects and social campaigns to daily newsletters—as well as events, sales, and marketing products. Collaborate closely with key staff to design and implement products for mobile, tablet, and desktop, from an extensive redesign to sales and marketing integrations....',
  'url': 'http://www.indeed.com/rc/clk?jk=8986a15fbb5aa6b8&atk=',
  'onmousedown': "indeed_clk(this,'');",
  'latitude': 40.73551,
  'longitude': -73.9919,
  'jobkey': '8986a15fbb5aa6b8',
  'sponsored': False,
  'expired': False,
  'indeedApply': True,
  'formattedLocationFull': 'New York, NY 10003',
  'formattedRelativeTime': '8 days ago',
  'stations': '',
  '

In [13]:
save_jobs_to_disk(indeed_jobs, os.path.join(data_dir, "indeed_jobs.json"))

#### The Muse API

In [14]:
def search_themuse_jobs(*, category, location=None, limit=None):
    """
    Args:
        category (str): Job category to search.
        location (str): Like "New York, NY".
        limit (int)
        
    Returns:
        List[dict]
    """
    if category not in _CATEGORIES:
        raise ValueError(
            "category={} is invalid; valid values are {}".format(category, _CATEGORIES))
    base_url = "https://www.themuse.com/api/public/jobs"
    params = {"category": category}
    if location:
        params["location"] = location
    results = []
    i = 0
    while True:
        page_params = params.copy()
        page_params["page"] = i
        try:
            response = requests.get(base_url, params=page_params)
            response.raise_for_status()
        except requests.RequestException as e:
            print(e)
            break
        data = response.json()
        page_results = data.get("results", [])
        results.extend(page_results)
        i += 1
        if len(page_results) < data["items_per_page"]:
            break
        elif limit is not None and len(results) > limit:
            break
    if limit is not None:
        return results[:limit]
    else:
        return results
    
    
_CATEGORIES = {
    "Account Management",
    "Business & Strategy",
    "Creative & Design",
    "Customer Service",
    "Data Science",
    "Editorial",
    "Education",
    "Engineering",
    "Finance",
    "Fundraising & Development",
    "Healthcare & Medicine",
    "HR & Recruiting",
    "Legal",
    "Marketing & PR",
    "Operations",
    "Project & Product Management",
    "Retail",
    "Sales",
    "Social Media & Community",
}

In [15]:
all_search_results = []
for category in category_queries:
    print("category:", category)
    try:
        results = search_themuse_jobs(category=category, location="New York, NY", limit=100)
        all_search_results.extend(results)
    except Exception:
        break

# deduplicate the list of search results
themuse_jobs = dedupe_results(all_search_results, "id")
print("# jobs fetched =", len(themuse_jobs))
themuse_jobs[:1]

category: Engineering
category: Data Science
# jobs fetched = 116


[{'contents': "<p><br><br>The Marquee team at Goldman Sachs delivers digital products and services to our institutional clients. We design and build highly-scalable solutions allowing access to Goldman Sachs' content, data, analytics, risk and execution services. These solutions transform client experiences while generating new revenue streams and business models. Our strategic initiatives disrupt the status-quo by innovating new businesses that unlock the power of Goldman Sachs for our clients. <br><br>Our team of engineers, designers, product managers and salespeople collaborate to drive progress within digital finance. We surround ourselves with big thinkers who thrive on tackling fresh challenges and uniting around a common vision. We move fast, embrace change and welcome fresh perspectives. Help us build the future of finance while positioned at the center of global markets. <br><br>Marquee Sales is accountable for all aspects of the digital sales cycle, including lead generation,

In [16]:
save_jobs_to_disk(themuse_jobs, os.path.join(data_dir, "themuse_jobs.json"))

#### combined dataset

(assuming no duplicates across sources)

In [17]:
print("total # of jobs fetched = ", len(github_jobs + indeed_jobs + themuse_jobs))

total # of jobs fetched =  579
